# Proyecto: Descifrando Mensajes Codificados Usando MCMC

Integrantes: Sebastián Flores y Matías Neto

## Procesamiento de la fuente de texto

In [2]:
%pip install pycipher

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 KB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycipher: filename=pycipher-0.5.2-py3-none-any.whl size=30456 sha256=9de64c40398958bad39dd55ec22b4d2f7946c1fe5d214d3fe7fe9420b2d0b1ac
  Stored in directory: /home/sflores/.cache/pip/wheels/ca/e7/37/bf758675337f9b98f096d8f7a5fd0cf320aadd67ae8a12f545
Successfully built pycipher
You should consider upgrading via the '/home/sflores/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [41]:
import re
import pandas as pd
import numpy as np
from pycipher import SimpleSubstitution
import random

In [24]:
# Cargar el texto
with open("gloria_benitoperezgaldos.txt", "r") as f:
    text = f.readlines()

for i in range(len(text)):
    text[i] = text[i].replace('\n', '')

text = ' '.join(text)

In [26]:
# Sólo minúsculas
clean_text = text.casefold()

# Eliminar tildes
tilde_dict = {'á':'a', 'é': 'e', 'í': 'i', 'ó':'o', 'ú':'u', 'ñ': 'n'}
for key, value in tilde_dict.items():
    clean_text = clean_text.replace(key, value)

# Eliminar caracteres especiales
clean_text = re.sub(r"[^a-zA-Z ]", "", clean_text)
clean_text = clean_text.upper()

In [40]:
def process(word, matrix):
    '''
    Almacena la frecuencia de transiciones de una palabra en una matriz
    
    Argumentos:
        word (string): Palabra a procesar
        matrix (pd.DataFrame): Matriz de transición
    
    Retorna:
        matrix (pd.DataFrame): Matriz de transición actualizada con
                               la palabra procesada
    '''
    letters = list(word)
    matrix.loc[' ', letters[0]] = matrix.loc[' ', letters[0]] + 1
    for i in range(len(word)-1):
        matrix.loc[letters[i], letters[i+1]] = matrix.loc[letters[i], letters[i+1]] + 1
    matrix.loc[letters[len(word)-1], ' '] = matrix.loc[letters[len(word)-1], ' '] + 1
    return matrix

In [28]:
# Creación de la matriz
alphabet = sorted(''.join(set(clean_text)))
matrix = pd.DataFrame(data=np.zeros((len(alphabet), len(alphabet))), index=alphabet, columns=alphabet)
words = clean_text.split()

# Se demora unos minutos
for word in words:
    matrix = process(word, matrix)
transition = matrix.apply(lambda x: x/np.sum(x), axis=1)

## Preparación de la función de costos

In [38]:
def plausibility(ciphertext, key, matrix = transition):
    '''
    Función de plausibilidad que funcionará como costo para SA
    
    Argumentos:
        ciphertext (string): Texto codificado
        key (string): Clave del mismo largo que el alfabeto a testear
        matrix (pd.DataFrame): Matriz de transición construida antes
    
    Retorna:
        plausibility (float): valor de la plausibilidad para key
    '''
    cipher = SimpleSubstitution(key)
    plaintext = cipher.decipher(ciphertext, keep_punct = True)
    words = plaintext.split()
    matrix = pd.DataFrame(data=np.zeros((len(alphabet), len(alphabet))), index=alphabet, columns=alphabet)
    for word in words:
        matrix = process(word, matrix)
    plausibility = np.product((1 + transition.to_numpy()) ** matrix.to_numpy())
    return plausibility

# Ejemplo
plaintext = "MUCHOS ANOS DESPUES FRENTE AL PELOTON DE FUSILAMIENTO " + \
            "EL CORONEL AURELIANO BUENDIA RECORDO AQUELLA TARDE REMOTA " + \
            "EN QUE SU PADRE LO LLEVO A CONOCER EL HIELO"
real_key = "AJPCZWRLFBDKOTYUQGENHXMIVS"
real_cipher = SimpleSubstitution(real_key)
ciphertext = real_cipher.encipher(plaintext, keep_punct = True)
print(f"Texto cifrado: {ciphertext}")
print(f"Plausibilidad: {plausibility(ciphertext, real_key)}")

Texto cifrado: OHPLYE ATYE CZEUHZE WGZTNZ AK UZKYNYT CZ WHEFKAOFZTNY ZK PYGYTZK AHGZKFATY JHZTCFA GZPYGCY AQHZKKA NAGCZ GZOYNA ZT QHZ EH UACGZ KY KKZXY A PYTYPZG ZK LFZKY
Plausibilidad: 11337837059.408495


In [39]:
# Ejemplo para ver que efectivamente la plausibilidad es menor
example_key = ''.join(random.sample(real_key, len(real_key)))
print("Clave ejemplo: " + example_key)
example_cipher = SimpleSubstitution(example_key)
print("Texto descifrado con la clave: "+example_cipher.decipher(ciphertext, keep_punct=True))
print(f"Plausibilidad: {plausibility(ciphertext, example_key)}")

Clave ejemplo: LHWIJDOGXUTPRVAFECSNQKYBZM
Texto descifrado con la clave: GBLAWQ OKWQ RYQJBYQ CHYKTY OV JYVWTWK RY CBQPVOGPYKTW YV LWHWKYV OBHYVPOKW EBYKRPO HYLWHRW OUBYVVO TOHRY HYGWTO YK UBY QB JORHY VW VVYIW O LWKWLYH YV APYVW
Plausibilidad: 19663.15526269065


## Preparación del paseo aleatorio

In [49]:
def neighbor(key):
    '''
    Retorna un vecino aleatorio (con distribución uniforme) de la clave
    
    Argumentos:
        key (str): Clave representada por una permutación del alfabeto
    
    Retorna:
        new_key (str): Clave vecina a key
    '''
    indices = random.sample(list(np.arange(len(key))), 2)
    symbol_a, symbol_b = key[indices[0]], key[indices[1]]
    new_key = list(key)
    new_key[indices[0]] = symbol_b
    new_key[indices[1]] = symbol_a
    new_key = ''.join(new_key)
    return new_key

nghbr = neighbor(example_key)
print(f"Clave ejemplo: {example_key}")
print(f"Vecino: {nghbr}")
print(f"Indices intercambiados: {[i for i in range(len(example_key)) if example_key[i] != nghbr[i]]}")

Clave ejemplo: LHWIJDOGXUTPRVAFECSNQKYBZM
Vecino: LHWIJDOKXUTPRVAFECSNQGYBZM
Indices intercambiados: [7, 21]
